In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('users.csv')


In [ ]:
df_repo = pd.read_csv('repositories.csv')

In [ ]:
df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,year,account_age,follower_ratio
0,ValentineFernandes,Valentine Fernandes,Unknown,"Mumbai, India",NaN,False,HTML | CSS | JS | SQL | MYSQL | JAVA,66,5247,5275,2022-01-29T08:11:37Z,2022,3,1749.000000
1,kovidgoyal,Kovid Goyal,Unknown,"Mumbai, India",NaN,False,Principal developer of calibre and kitty,37,4275,0,2012-01-06T05:38:24Z,2012,13,328.846154
2,slidenerd,slidenerd,SLIDENERD,Mumbai,slidenerd@gmail.com,True,"Bots, AI, advanced web frameworks, ohlc applic...",113,3166,1,2013-08-01T14:17:19Z,2013,12,263.833333
3,aryashah2k,Arya Shah,OPENAOD,"Mumbai, India",NaN,False,Computer Science Major | Machine Learning | So...,88,2595,2580,2020-11-03T03:06:19Z,2020,5,519.000000
4,coding-parrot,Gaurav Sen,INTERVIEWREADY,"Mumbai, India",NaN,False,CEO of InterviewReady,11,2411,0,2020-01-03T14:13:35Z,2020,5,482.200000


In [ ]:
df_repo.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,ValentineFernandes,ValentineFernandes/Food-Website,2022-06-01T17:20:57Z,54,54,HTML,True,True,MIT License
1,ValentineFernandes,ValentineFernandes/DentalClinic-Website,2022-06-02T10:28:55Z,59,59,HTML,True,True,MIT License
2,ValentineFernandes,ValentineFernandes/ValentineFernandes,2022-02-12T09:27:51Z,59,59,NaN,True,False,NaN
3,ValentineFernandes,ValentineFernandes/PetDog-Complete-Website,2022-03-24T07:17:59Z,55,55,HTML,True,True,MIT License
4,ValentineFernandes,ValentineFernandes/MySQL-Assignment,2022-10-10T13:59:53Z,12,12,NaN,True,True,MIT License


# Questions

Q1. Who are the top 5 users in Mumbai with the highest number of followers? List their login in order, comma-separated.

In [ ]:
popular_logins = list(df.sort_values(by='followers', ascending=False).head()['login'])

In [ ]:
for login in popular_logins:
    print(login, end=',')

ValentineFernandes,kovidgoyal,slidenerd,aryashah2k,coding-parrot,

Q2. Who are the 5 earliest registered GitHub users in Mumbai? List their login in ascending order of created_at, comma-separated.

In [ ]:
early_logins = list(df.sort_values(by='created_at').head()['login'])

In [ ]:
for login in early_logins:
    print(login, end=',')

ivank,sandeepshetty,svs,nitinhayaran,nischal,

Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [ ]:
pop_liscense = list(df_repo.value_counts('license_name').head(n=3).index)

In [ ]:
for license in pop_liscense:
    print(license, end=',')

MIT License,Apache License 2.0,Other,

Q4. Which company do the majority of these developers work at?

In [ ]:
df['company'].value_counts().head(n=1).index[0]

'MASAI SCHOOL'

Q5. Which programming language is most popular among these users?

In [ ]:
df_repo['language'].value_counts().head(n=1).index[0]

'JavaScript'

Q6. Which programming language is the second most popular among users who joined after 2020?

In [ ]:
def compare_dates(date):
    if int(date.split('-')[0]) > 2020:
      return True
    else:
      return False


In [ ]:
df['after_2020'] = df['created_at'].apply(compare_dates)

In [ ]:
after_2020 = list(df[df['after_2020'] == True]['login'])

In [ ]:
def after_2020_language(login):
    if login in after_2020:
        return True
    else:
        return False

In [ ]:
df_repo['after_2020'] = df_repo['login'].apply(after_2020_language)

In [ ]:
df_repo[df_repo['after_2020']]['language'].value_counts().head(n=2).index[-1]

'HTML'

Q7. Which language has the highest average number of stars per repository?

In [ ]:
df_repo.groupby('language').mean('stargazers_count').sort_values(by='stargazers_count', ascending=False).index[0]

'TSQL'

Q8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [ ]:
df['leader_strength'] = df['followers'] / (1 + df['following'])

In [ ]:
leaders = list(df.sort_values(by='leader_strength', ascending=False).head()['login'])

In [ ]:
for leader in leaders:
  print(leader, end=',')

kovidgoyal,coding-parrot,gkcs,slidenerd,dmalvia,

Q9. What is the correlation between the number of followers and the number of public repositories among users in Mumbai?

In [ ]:
df[['followers', 'public_repos']].corr()

,followers,public_repos
followers,1.000000,0.035862
public_repos,0.035862,1.000000


Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [ ]:
lin_reg = LinearRegression()

In [ ]:
X = df['public_repos']
y = df['followers']

In [ ]:
lin_reg.fit(X.values.reshape(-1, 1), y)

LinearRegression()

In [ ]:
lin_reg.coef_

array([0.10478927])

Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [ ]:
df_repo[['has_projects', 'has_wiki']].corr()

,has_projects,has_wiki
has_projects,1.000000,0.158327
has_wiki,0.158327,1.000000


Q12. Do hireable users follow more people than those who are not hireable?

Average of following per user for hireable=true minus the average following for the rest (to 3 decimal places, e.g. 12.345 or -12.345)

In [ ]:
df.groupby('hireable')['following'].mean()

,following
hireable,
False,117.828947
True,126.157706


In [ ]:
126.157706 - 117.828947

8.328759000000005

Q13. Some developers write long bios. Does that help them get more followers?

What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)
Regression slope of followers on bio word count (to 3 decimal places, e.g. 12.345 or -12.345)

In [ ]:
df.iloc[695]['bio']

'I Hack for Express not for Impress 🔮🌟😳\r\nShizo The Techie'

In [ ]:
df['bio']

,bio
0,HTML | CSS | JS | SQL | MYSQL | JAVA
1,Principal developer of calibre and kitty
2,"Bots, AI, advanced web frameworks, ohlc applic..."
3,Computer Science Major | Machine Learning | So...
4,CEO of InterviewReady
...,...
730,"Coding enthusiast, interested in everything te..."
731,Coding My Way Through Life!\r\n
732,Web Developer & qlik sense developer
733,Aspiring Software Engineer.\r\n


In [ ]:
df['bio_length'] = df['bio'].apply(lambda x: len(str(x).strip().split()) if str(x) != "" else 0)

In [ ]:
df[['bio_length', 'followers']].corr()

,bio_length,followers
bio_length,1.000000,0.017732
followers,0.017732,1.000000


In [ ]:
df[df['bio_length'] != 0][['bio_length', 'followers']].corr()

,bio_length,followers
bio_length,1.000000,-0.000875
followers,-0.000875,1.000000


In [ ]:
df_with_bios = df[df['bio'].notna()]

# Step 2: Calculate bio length (in words)
df_with_bios['bio_length'] = df_with_bios['bio'].apply(lambda x: len(x.split()))

# Step 3: Calculate the correlation (optional if only the regression slope is needed)
correlation = df_with_bios[['bio_length', 'followers']].corr().iloc[0, 1]
print(f"Correlation between bio length and followers: {correlation:.4f}")

# Step 4: Calculate the regression slope of followers on bio word count
slope, intercept, r_value, p_value, std_err = linregress(df_with_bios['bio_length'], df_with_bios['followers'])
print(f"Regression slope of followers on bio word count: {slope:.4f}")

Correlation between bio length and followers: 0.0177
Regression slope of followers on bio word count: 0.7647


In [ ]:
df_13 = df[df['bio'] != ""]

In [ ]:
df[['bio_length', 'followers']].corr()

,bio_length,followers
bio_length,1.000000,0.017732
followers,0.017732,1.000000


In [ ]:
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(df['bio_length'], df['followers'])

In [ ]:
slope

0.7647010633840894

In [ ]:
lin_model_2 = LinearRegression()

In [ ]:
X = df_13['bio_length']
y = df_13['followers']

In [ ]:
lin_model_2.fit(X.values.reshape(-1, 1), y)

LinearRegression()

In [ ]:
lin_model_2.coef_

array([-0.04260098])

Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [ ]:
def is_weekend(date):
    year = date.split('-')[0]
    month = date.split('-')[1]
    day = date.split('-')[2][:2]

    date = datetime(int(year), int(month), int(day))
    if date.weekday() >= 5:
        return True
    else:
        return False

In [ ]:
df_repo['weekend'] = df_repo['created_at'].apply(is_weekend)

In [ ]:
weekend_logins = df_repo[df_repo['weekend'] == True]['login'].value_counts().head().index

In [ ]:
for login in weekend_logins:
    print(login, end=',')

Kushal334,alokproc,vinod1988,patilswapnilv,rajeshpillai,

Q15. Do people who are hireable share their email addresses more often?


[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

In [ ]:
df[(df['hireable'] == True) & (df['email'].notna())].shape[0] / df[(df['hireable'] == True)].shape[0]

0.6057347670250897

In [ ]:
df[(df['hireable'] == False) & (df['email'].notna())].shape[0] / df[df['hireable'] == False].shape[0]

0.375

In [ ]:
df[(df['email'].notna())].shape[0] / df.shape[0]

0.46258503401360546

In [ ]:
0.6057347670250897 - 0.375

0.23073476702508966

In [ ]:
0.6057347670250897 - 0.46258503401360546

0.1431497330114842

Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [ ]:
def get_surname(name):
    if name is not None:
        return str(name).strip().split()[-1]
    else:
        return None

In [ ]:
df['surname'] = df['name'].apply(get_surname)

In [ ]:
# trim whiespace from v

'asasas   asasas asas asas'

In [ ]:
df['surname'].value_counts().head()

,count
surname,
Singh,17
Shah,15
Yadav,13
Shaikh,12
Sharma,11


#README Prep

In [ ]:
df.head()


,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,ValentineFernandes,Valentine Fernandes,NaN,"Mumbai, India",NaN,False,HTML | CSS | JS | SQL | MYSQL | JAVA,66,5247,5275,2022-01-29T08:11:37Z
1,kovidgoyal,Kovid Goyal,NaN,"Mumbai, India",NaN,False,Principal developer of calibre and kitty,37,4275,0,2012-01-06T05:38:24Z
2,slidenerd,slidenerd,SLIDENERD,Mumbai,slidenerd@gmail.com,True,"Bots, AI, advanced web frameworks, ohlc applic...",113,3166,1,2013-08-01T14:17:19Z
3,aryashah2k,Arya Shah,OPENAOD,"Mumbai, India",NaN,False,Computer Science Major | Machine Learning | So...,88,2595,2580,2020-11-03T03:06:19Z
4,coding-parrot,Gaurav Sen,INTERVIEWREADY,"Mumbai, India",NaN,False,CEO of InterviewReady,11,2411,0,2020-01-03T14:13:35Z


In [ ]:
df['company'].fillna('Unknown', inplace=True)

In [ ]:
df[["hireable", 'public_repos', "followers", "following"]].corr()['followers']

,followers
hireable,-0.016775
public_repos,0.035862
followers,1.000000
following,0.400148


In [ ]:
df[df['hireable']][['public_repos', "followers", "following"]].corr()

,public_repos,followers,following
public_repos,1.000000,0.047681,0.495451
followers,0.047681,1.000000,0.057690
following,0.495451,0.057690,1.000000


In [ ]:
no_job_df = df[df['company'].isna()]

In [ ]:
job_df = df[df['company'].notna()]

In [ ]:
print(no_job_df['public_repos'].mean())
print(job_df['public_repos'].mean())

53.909937888198755
66.32203389830508


In [ ]:
print(no_job_df['followers'].mean())
print(job_df['followers'].mean())

149.6832298136646
158.65133171912834


In [ ]:
print(no_job_df['following'].mean())
print(job_df['following'].mean())

115.87267080745342
124.98062953995158


In [ ]:
df['company'].value_counts()

,count
company,
Unknown,322
MASAI SCHOOL,14
BROWSERSTACK,12
IIT BOMBAY,9
FRAPPE,6
...,...
WEBFUN,1
FREELANCE .NET DEVELOPER,1
EPICGAMES SML-INC,1


In [ ]:
df.groupby('company')['followers'].mean().sort_values(ascending=False)

,followers
company,
SLIDENERD,3166.0
OPENAOD,2595.0
INTERVIEWREADY,2411.0
DIRECTI,2178.0
PRACTO TOKOPEDIA,1306.0
...,...
B2X,50.0
UNIVERSITY OF MUMBAI,50.0
RELIANCE JIO,50.0


In [ ]:
df['year'] = df['created_at'].apply(lambda x: x.split('-')[0])

In [ ]:
df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,year
0,ValentineFernandes,Valentine Fernandes,Unknown,"Mumbai, India",NaN,False,HTML | CSS | JS | SQL | MYSQL | JAVA,66,5247,5275,2022-01-29T08:11:37Z,2022
1,kovidgoyal,Kovid Goyal,Unknown,"Mumbai, India",NaN,False,Principal developer of calibre and kitty,37,4275,0,2012-01-06T05:38:24Z,2012
2,slidenerd,slidenerd,SLIDENERD,Mumbai,slidenerd@gmail.com,True,"Bots, AI, advanced web frameworks, ohlc applic...",113,3166,1,2013-08-01T14:17:19Z,2013
3,aryashah2k,Arya Shah,OPENAOD,"Mumbai, India",NaN,False,Computer Science Major | Machine Learning | So...,88,2595,2580,2020-11-03T03:06:19Z,2020
4,coding-parrot,Gaurav Sen,INTERVIEWREADY,"Mumbai, India",NaN,False,CEO of InterviewReady,11,2411,0,2020-01-03T14:13:35Z,2020


In [ ]:
df['year'].value_counts().sort_index()

,count
year,
2008,2
2009,9
2010,7
2011,22
2012,22
2013,21
2014,45
2015,39
2016,50


In [ ]:
df['account_age'] = df["year"].apply(lambda x: 2025 - int(x))

In [ ]:
df['account_age']

,count
account_age,
6,99
5,96
3,83
4,72
7,72
8,68
9,50
11,45
10,39


In [ ]:
df['follower_ratio'] = df['followers'] / df['account_age']

In [ ]:
df.groupby("year")["follower_ratio"].mean()

,follower_ratio
year,
2008,3.823529
2009,10.201389
2010,10.933333
2011,11.441558
2012,24.569930
2013,24.988095
2014,14.092929
2015,19.115385
2016,18.624444


In [ ]:
df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,year,account_age,follower_ratio
0,ValentineFernandes,Valentine Fernandes,Unknown,"Mumbai, India",NaN,False,HTML | CSS | JS | SQL | MYSQL | JAVA,66,5247,5275,2022-01-29T08:11:37Z,2022,3,1749.000000
1,kovidgoyal,Kovid Goyal,Unknown,"Mumbai, India",NaN,False,Principal developer of calibre and kitty,37,4275,0,2012-01-06T05:38:24Z,2012,13,328.846154
2,slidenerd,slidenerd,SLIDENERD,Mumbai,slidenerd@gmail.com,True,"Bots, AI, advanced web frameworks, ohlc applic...",113,3166,1,2013-08-01T14:17:19Z,2013,12,263.833333
3,aryashah2k,Arya Shah,OPENAOD,"Mumbai, India",NaN,False,Computer Science Major | Machine Learning | So...,88,2595,2580,2020-11-03T03:06:19Z,2020,5,519.000000
4,coding-parrot,Gaurav Sen,INTERVIEWREADY,"Mumbai, India",NaN,False,CEO of InterviewReady,11,2411,0,2020-01-03T14:13:35Z,2020,5,482.200000


In [ ]:
df_repo.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,ValentineFernandes,ValentineFernandes/Food-Website,2022-06-01T17:20:57Z,54,54,HTML,True,True,MIT License
1,ValentineFernandes,ValentineFernandes/DentalClinic-Website,2022-06-02T10:28:55Z,59,59,HTML,True,True,MIT License
2,ValentineFernandes,ValentineFernandes/ValentineFernandes,2022-02-12T09:27:51Z,59,59,NaN,True,False,NaN
3,ValentineFernandes,ValentineFernandes/PetDog-Complete-Website,2022-03-24T07:17:59Z,55,55,HTML,True,True,MIT License
4,ValentineFernandes,ValentineFernandes/MySQL-Assignment,2022-10-10T13:59:53Z,12,12,NaN,True,True,MIT License


In [ ]:
df_lang = df_repo.groupby('login')['language'].agg(lambda x: x.mode())

In [ ]:
df_lang.values

array(['Java', 'Python', array(['CSS', 'Java', 'Python'], dtype=object),
       'Python', 'Python', 'JavaScript', 'C#', 'HTML', 'Python',
       'JavaScript', 'JavaScript', 'C++', 'JavaScript', 'TypeScript', 'C',
       'Dart', array(['Java', 'Python'], dtype=object), 'Java',
       'Jupyter Notebook', 'JavaScript', 'Python', 'JavaScript',
       'JavaScript', 'JavaScript', 'JavaScript',
       array(['Java', 'JavaScript'], dtype=object), 'Jupyter Notebook',
       'Jupyter Notebook', 'Ruby', 'JavaScript', 'Jupyter Notebook',
       'JavaScript', 'Python',
       array(['HTML', 'JavaScript'], dtype=object), 'JavaScript',
       'JavaScript', 'Python', 'JavaScript', 'HTML', 'HTML', 'Ruby',
       'HTML', 'Shell', 'Jupyter Notebook', 'Python', 'JavaScript',
       'Jupyter Notebook', 'JavaScript', 'TypeScript', 'Java',
       'Jupyter Notebook', 'JavaScript', 'Dart', 'Java', 'Python',
       'JavaScript', 'C', 'Java', 'C', 'Jupyter Notebook', 'JavaScript',
       'Jupyter Notebook', 'PHP

In [ ]:
df_2 = pd.DataFrame(df_lang.values, columns=['language'])

In [ ]:
df_2['login'] = df_lang.index

In [ ]:
df_2['login']

,login
0,1407arjun
1,5arthak01
2,9vishwashah
3,ARYANK-08
4,Aafiya-H
...,...
726,zaidmukaddam
727,zemse
728,zoe-abedi
729,zoneAnaS


In [ ]:
df['lang'] = df['login'].apply(foll)

HTML
Python
JavaScript
Jupyter Notebook
Java
Java
Jupyter Notebook
Jupyter Notebook
JavaScript
JavaScript
['JavaScript' 'Python']
C++
JavaScript
C++
Java
JavaScript
JavaScript
HTML
Python
JavaScript
Java
JavaScript
['C#' 'JavaScript']
TypeScript
Python
Python
JavaScript
Shell
Jupyter Notebook
Jupyter Notebook
Shell
PHP
Jupyter Notebook
Java
Python
Java
Python
['HTML' 'JavaScript' 'Python']
Python
TypeScript
Python
C
JavaScript
Swift
C
HTML
['C++' 'Python']
HTML
JavaScript
JavaScript
['C++' 'HTML']
JavaScript
JavaScript
['Jupyter Notebook' 'Python']
Ruby
JavaScript
['Go' 'Kotlin']
Java
JavaScript
C
Python
JavaScript
JavaScript
TypeScript
JavaScript
Python
Python
['JavaScript' 'Python']
Jupyter Notebook
Python
JavaScript
JavaScript
PHP
Dart
Swift
Jupyter Notebook
Jupyter Notebook
JavaScript
Python
JavaScript
Dart
HTML
JavaScript
JavaScript
JavaScript
Java
['Jupyter Notebook' 'Python']
Python
JavaScript
Java
Java
C++
Python
Java
Shell
Java
JavaScript
Jupyter Notebook
Python
HTML
Jupyter N

AttributeError: 'builtin_function_or_method' object has no attribute 'get_indexer'

In [ ]:
def foll(login):
  try:
    lang = df_2[df_2['login'] == login]['language']
    print(lang.iloc[0])

    return lang
  except:
    return ""

In [ ]:
merged_df = pd.merge(df, df_2, on='login', how='inner')

In [ ]:
merged_df.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,year,account_age,follower_ratio,language
0,ValentineFernandes,Valentine Fernandes,Unknown,"Mumbai, India",NaN,False,HTML | CSS | JS | SQL | MYSQL | JAVA,66,5247,5275,2022-01-29T08:11:37Z,2022,3,1749.000000,HTML
1,kovidgoyal,Kovid Goyal,Unknown,"Mumbai, India",NaN,False,Principal developer of calibre and kitty,37,4275,0,2012-01-06T05:38:24Z,2012,13,328.846154,Python
2,slidenerd,slidenerd,SLIDENERD,Mumbai,slidenerd@gmail.com,True,"Bots, AI, advanced web frameworks, ohlc applic...",113,3166,1,2013-08-01T14:17:19Z,2013,12,263.833333,JavaScript
3,aryashah2k,Arya Shah,OPENAOD,"Mumbai, India",NaN,False,Computer Science Major | Machine Learning | So...,88,2595,2580,2020-11-03T03:06:19Z,2020,5,519.000000,Jupyter Notebook
4,coding-parrot,Gaurav Sen,INTERVIEWREADY,"Mumbai, India",NaN,False,CEO of InterviewReady,11,2411,0,2020-01-03T14:13:35Z,2020,5,482.200000,Java


In [ ]:
merged_df['language'].value_counts()

,count
language,
JavaScript,253
Python,139
Jupyter Notebook,73
Java,57
HTML,38
...,...
MATLAB,1
[],1
"[Java, Python]",1


In [ ]:
merged_df['language'] = merged_df['language'].apply(lambda x: str(x) if type(x) == np.ndarray else str(x))

In [ ]:
merged_df.groupby('language')['followers'].mean().sort_values(ascending=False)

,followers
language,
['C#' 'JavaScript'],529.000000
['HTML' 'JavaScript' 'Python'],381.000000
['C++' 'Python'],315.000000
['C++' 'HTML'],300.000000
['Go' 'Kotlin'],291.000000
['Jupyter Notebook' 'Python'],259.500000
HTML,256.842105
C++,230.400000
Java,210.403509


In [ ]:
merged_df.groupby('hireable')['language'].value_counts()

hireable  language                     
False     JavaScript                       150
          Python                            86
          Jupyter Notebook                  44
          Java                              38
          HTML                              30
                                          ... 
True      ['HTML' 'JavaScript' 'Vue']        1
          ['HTML' 'JavaScript']              1
          ['Java' 'Python']                  1
          ['JavaScript' 'TypeScript']        1
          ['Jupyter Notebook' 'Python']      1
Name: count, Length: 77, dtype: int64

In [ ]:
merged_df[merged_df['company'] != "Unknown"]["language"].value_counts()

,count
language,
JavaScript,148
Python,83
Jupyter Notebook,37
Java,32
HTML,17
TypeScript,12
Dart,7
PHP,7
C,6


In [ ]:
merged_df[merged_df['company'] == "Unknown"]["language"].value_counts()

,count
language,
JavaScript,105
Python,56
Jupyter Notebook,36
Java,25
HTML,21
TypeScript,10
Dart,8
C,6
C++,4


In [ ]:
merged_df['company'].value_counts()

,count
company,
Unknown,318
MASAI SCHOOL,14
BROWSERSTACK,12
IIT BOMBAY,9
FRAPPE,6
...,...
WEBFUN,1
FREELANCE .NET DEVELOPER,1
EPICGAMES SML-INC,1


In [ ]:
The average number of followers of users who use C++ for the majority of thier projects is the highest, while C++ is not even in the 5 most common languages

In [ ]:
merged_df.groupby('year')['language'].value_counts().groupby(level=0).idxmax()

,count
year,
2008,"(2008, JavaScript)"
2009,"(2009, JavaScript)"
2010,"(2010, JavaScript)"
2011,"(2011, JavaScript)"
2012,"(2012, Python)"
2013,"(2013, JavaScript)"
2014,"(2014, Python)"
2015,"(2015, Python)"
2016,"(2016, Python)"
